HuggingFace is a company with a heavy open source philosophy that makes transformers readily available so you don't have to do what we did before for every application.

## Prep


In [1]:
!pip install -U datasets evaluate transformers transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:

import torch
import numpy as np
import random
import os

def set_seeds(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True

set_seeds()


## Tokenizing and loading the dataset

In HuggingFace there are many models, and each has its own tokenizer. Lucky for us there is a class `AutoTokenizer` that does the heavylifting after we provide a checkpoint

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

raw_datasets = load_dataset("imdb")  # load imdb dataset
raw_datasets


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Notice it is a dict object with the train, test, and unsupervised datasets to play around

In [4]:
raw_datasets['train'][0]  # Let's see the first review

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [5]:
raw_datasets['train'].features


{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

There it is, within features we see that the index 0 is **Negative**

Now to tokenise the dataset we need to load the proper tokenizer for the model we care about. And then we are going to apply it everywhere!

After this step the tokenizer converts the text into a Tensor of ids, each representing a different word in the BERT vocabulary

In [6]:

from transformers import AutoModelForSequenceClassification

model_name = "google/bert_uncased_L-2_H-128_A-2"  # Example model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The tokenizer from BERT (well DistillBERT) converts each word into its ID according to *its* vocabulary. And notice the masking says we haven't been truncated. What we will do know is do this for all data and convert it into a TF Datasets object (which Keras accepts)


In [7]:

from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from torch.optim import AdamW

# Ensure correct tokenization
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512, return_tensors="pt")

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
# Data collation
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# DataLoader setup
train_dataloader = DataLoader(tokenized_datasets["train"].select(range(1000)), shuffle=True, batch_size=32, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["test"].select(range(1000)), batch_size=32, collate_fn=data_collator)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
# Inspect DataLoader output
for batch in train_dataloader:
    print("Batch shapes:")
    for key, value in batch.items():
        if hasattr(value, 'shape'):
            print(f"{key}: {value.shape}")
        else:
            print(f"{key}: {type(value)} - shape attribute not found")
    break  # Remove this break statement to inspect more batches


Batch shapes:
labels: torch.Size([32])
input_ids: torch.Size([32, 512])
token_type_ids: torch.Size([32, 512])
attention_mask: torch.Size([32, 512])


# Fit the model

In [9]:
from torch.optim import AdamW
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(15):
    model.train()
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    model.eval()
    total_eval_loss = 0
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        total_eval_loss += outputs.loss.item()
    avg_eval_loss = total_eval_loss / len(eval_dataloader)
    print(f"Average evaluation loss: {avg_eval_loss}")


  0%|          | 0/32 [00:00<?, ?it/s]

Average evaluation loss: 0.2237233119085431


  0%|          | 0/32 [00:00<?, ?it/s]

Average evaluation loss: 0.14380764635279775


  0%|          | 0/32 [00:00<?, ?it/s]

...

Average evaluation loss: 0.015352412214269862


  0%|          | 0/32 [00:00<?, ?it/s]

Average evaluation loss: 0.01376337424153462


  0%|          | 0/32 [00:00<?, ?it/s]

Average evaluation loss: 0.01242850540438667
